This notebook process the previously obtained dataframe in order to better organize data and obtaining up-to-date descriptions and cover images. Pictures are downloaded from the URLs of each podcast, stored in a Google Drive folder and then zipped in order to save space.  

The final dataset contains 29279 different podcast, each of them described in terms of its title, authors' names, primary genre (the label necessary for training) and its cover image.

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/all_podcast.csv')

In [ ]:
df

,authors,Name,Artwork,Genre IDs,Primary Genre,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description
0,Firearms Radio Network,Firearms Radio Network (All Shows),https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['Wilderness', 'Podcasts', 'Sports', 'Leisure'...",Wilderness,0,0,https://podcasts.apple.com/us/podcast/firearms...,http://feeds.feedburner.com/FRNetwork,0,Master Feed
1,Jacob Mchangama,Clear and Present Danger - A history of free s...,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['History', 'Podcasts', 'Society & Culture']",History,0,0,https://podcasts.apple.com/us/podcast/clear-an...,https://freespeechhistory.libsyn.com/rss,0,"Why have kings, emperors, and governments kill..."
2,Kyric McMillan and Nick Vassolo,2 Idiots and a List,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['Music Commentary', 'Podcasts', 'Music']",Music Commentary,0,0,https://podcasts.apple.com/us/podcast/2-idiots...,https://anchor.fm/s/4a47e010/podcast/rss,0,"Welcome to “2 Idiots and a List”, the podcast ..."
3,Kate Spencer & Doree Shafrir,Forever35,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"['Fashion & Beauty', 'Podcasts', 'Arts', 'Heal...",Fashion & Beauty,0,0,https://podcasts.apple.com/us/podcast/forever3...,https://rss.acast.com/forever35,0,Which night creams will make you look well-res...
4,Maverick Alexander,DarbyCast,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"['Comedy Fiction', 'Podcasts', 'Fiction']",Comedy Fiction,0,0,https://podcasts.apple.com/us/podcast/darbycas...,https://anchor.fm/s/1c855d9c/podcast/rss,0,"Neither the Darbyshire book series, nor this p..."
...,...,...,...,...,...,...,...,...,...,...,...
31245,NaN,"MTG Pro Tutor - Insights, Tips & Advice from M...",https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['1461', '26', '1323', '1460']",Games,229,"[55.63, 48.43, 61.8, 52.75, 43.17, 42.68, 57.5...",https://itunes.apple.com/us/podcast/mtg-pro-tu...,http://mtgprotutor.libsyn.com/rss,http://www.mtgprotutor.com,MTG Pro Tutor is a top rated Magic: The Gather...
31246,NaN,National CASA Podcast,https://is4-ssl.mzstatic.com/image/thumb/Featu...,"['1475', '26', '1325', '1304', '1468']",Government,100,"[25.33, 26.57, 31.58, 2.23, 16.13, 1.72, 1.92,...",https://itunes.apple.com/us/podcast/national-c...,http://feeds.feedburner.com/NationalCasaPodcast,http://www.casaforchildren.org,"Today, more than 1,000 CASA program offices op..."
31247,NaN,CAR DISCOVERY｜PodcastLife,0,"['1454', '26', '1323', '1460', '1324', '1320',...",0,54,"[2.72, 2.72, 2.72, 2.55, 2.55, 2.55, 2.55, 2.5...",https://itunes.apple.com/us/podcast/car-discov...,http://feeds2.feedburner.com/CarDiscovery,http://www.dandl.co.jp/car-discovery/,【www.adelcars.com】アデル・カーズ株式会社提供番組：CAR DISCOVER...
31248,NaN,Tottenham Hotspur Family,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['1467', '26', '1316']",Sports,112,"[64.53, 60.5, 76.18, 74.55, 88.47, 75.98, 73.5...",https://itunes.apple.com/us/podcast/tottenham-...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/javad-movahedi-1,The Tottenham Hotspur Family Podcast\n\n\nA gl...


Some podcast of the old dataset does not have a primary genre; we can use the genre IDs, transform them in text and pick one of them (the first one) to fill the Primary Genre column.

In [ ]:
genre_ids= [['1311', 'News & Politics'], ['26', 'Podcasts'], ['1479', 'Social Sciences'], ['1315', 'Science & Medicine'],
            ['1324', 'Society & Culture'], ['1302', 'Personal Journals'], ['1469', 'Language Courses'], ['1304', 'Education'], 
            ['1320', 'Places & Travel'], ['1416', 'Higher Education'], ['1465', 'Professional'], ['1316', 'Sports & Recreation'],
            ['1303', 'Comedy'], ['1305', 'Kids & Family'], ['1439', 'Christianity'], ['1314', 'Religion & Spirituality'], 
            ['1444', 'Spirituality'], ['1309', 'TV & Film'], ['1462', 'History'], ['1310', 'Music'], ['1478', 'Medicine'], 
            ['1321', 'Business'], ['1412', 'Investing'], ['1420', 'Self-Help'], ['1307', 'Health'], ['1481', 'Alternative Health'], 
            ['1417', 'Fitness & Nutrition'], ['1467', 'Amateur'], ['1480', 'Software How-To'], ['1318', 'Technology'], ['1448', 'Tech News'], 
            ['1456', 'Outdoor'], ['1477', 'Natural Sciences'], ['1301', 'Arts'], ['1454', 'Automotive'], ['1323', 'Games & Hobbies'], 
            ['1438', 'Buddhism'], ['1443', 'Philosophy'], ['1401', 'Literature'], ['1402', 'Design'], ['1410', 'Careers'], ['1470', 'Training'],
            ['1413', 'Management & Marketing'], ['1306', 'Food'], ['1406', 'Visual Arts'], ['1446', 'Gadgets'], ['1468', 'Educational Technology'],
            ['1405', 'Performing Arts'], ['1460', 'Hobbies'], ['1471', 'Business News'], ['1404', 'Video Games'], ['1450', 'Podcasting'], 
            ['1473', 'National'], ['1325', 'Government & Organizations'], ['1461', 'Other Games'], ['1466', 'College & High School'], 
            ['1459', 'Fashion & Beauty'], ['1476', 'Non-Profit'], ['1415', 'K-12'], ['1455', 'Aviation'], ['1464', 'Other'], 
            ['1421', 'Sexuality'], ['1472', 'Shopping'], ['1475', 'Local'], ['1441', 'Judaism'], ['1440', 'Islam'],
            ['1474', 'Regional'], ['1463', 'Hinduism']]
df_noPrimeGenre = df[df['Primary Genre'] == '0']

In [ ]:
for index, row in df_noPrimeGenre.iterrows():
  new_genres = []
  g_ids = row['Genre IDs'].strip('][').split(', ')
  for el in g_ids:
    el = el.strip("'")
    for genre in genre_ids:
      if el in genre:
        new_genres.append(genre[1])
  df.loc[index, 'Genre IDs'] =str(new_genres)
  df.loc[index, 'Primary Genre'] = new_genres[0]


In [ ]:
df = df[df['Artwork'] != '0']
df.to_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/all_podcast.csv', index = False) #save df with update genre IDs and all primary Genre

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/all_podcast.csv')
df

,authors,Name,Artwork,Genre IDs,Primary Genre,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description
0,Firearms Radio Network,Firearms Radio Network (All Shows),https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['Wilderness', 'Podcasts', 'Sports', 'Leisure'...",Wilderness,0,0,https://podcasts.apple.com/us/podcast/firearms...,http://feeds.feedburner.com/FRNetwork,0,Master Feed
1,Jacob Mchangama,Clear and Present Danger - A history of free s...,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['History', 'Podcasts', 'Society & Culture']",History,0,0,https://podcasts.apple.com/us/podcast/clear-an...,https://freespeechhistory.libsyn.com/rss,0,"Why have kings, emperors, and governments kill..."
2,Kyric McMillan and Nick Vassolo,2 Idiots and a List,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['Music Commentary', 'Podcasts', 'Music']",Music Commentary,0,0,https://podcasts.apple.com/us/podcast/2-idiots...,https://anchor.fm/s/4a47e010/podcast/rss,0,"Welcome to “2 Idiots and a List”, the podcast ..."
3,Kate Spencer & Doree Shafrir,Forever35,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"['Fashion & Beauty', 'Podcasts', 'Arts', 'Heal...",Fashion & Beauty,0,0,https://podcasts.apple.com/us/podcast/forever3...,https://rss.acast.com/forever35,0,Which night creams will make you look well-res...
4,Maverick Alexander,DarbyCast,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"['Comedy Fiction', 'Podcasts', 'Fiction']",Comedy Fiction,0,0,https://podcasts.apple.com/us/podcast/darbycas...,https://anchor.fm/s/1c855d9c/podcast/rss,0,"Neither the Darbyshire book series, nor this p..."
...,...,...,...,...,...,...,...,...,...,...,...
29274,NaN,Within the Trenches,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"['1304', '26']",Education,157,0,https://itunes.apple.com/us/podcast/within-the...,http://www.thejabberlog.com/category/within-th...,http://www.thejabberlog.com/category/within-th...,The JCast on Jabber Log features two main show...
29275,NaN,"MTG Pro Tutor - Insights, Tips & Advice from M...",https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['1461', '26', '1323', '1460']",Games,229,"[55.63, 48.43, 61.8, 52.75, 43.17, 42.68, 57.5...",https://itunes.apple.com/us/podcast/mtg-pro-tu...,http://mtgprotutor.libsyn.com/rss,http://www.mtgprotutor.com,MTG Pro Tutor is a top rated Magic: The Gather...
29276,NaN,National CASA Podcast,https://is4-ssl.mzstatic.com/image/thumb/Featu...,"['1475', '26', '1325', '1304', '1468']",Government,100,"[25.33, 26.57, 31.58, 2.23, 16.13, 1.72, 1.92,...",https://itunes.apple.com/us/podcast/national-c...,http://feeds.feedburner.com/NationalCasaPodcast,http://www.casaforchildren.org,"Today, more than 1,000 CASA program offices op..."
29277,NaN,Tottenham Hotspur Family,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['1467', '26', '1316']",Sports,112,"[64.53, 60.5, 76.18, 74.55, 88.47, 75.98, 73.5...",https://itunes.apple.com/us/podcast/tottenham-...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/javad-movahedi-1,The Tottenham Hotspur Family Podcast\n\n\nA gl...


Next step is to download all the album cover from the URL and save them in a folder.

In [ ]:
import requests
import re
cover_links = df['Artwork'].tolist()
url_list = df['iTunes URL'].tolist()
file_paths = []
for index, link in enumerate(cover_links):
  url = url_list[index]
  matches = re.search(r"id\d+", url, re.MULTILINE)
  if matches != None:
    id = matches.group()
  else:
    id = 'None'
  filename = '/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/images/' +id+'.jpg'
  file_paths.append(filename)
  with open(filename, 'wb') as handle:
      response = requests.get(link, stream=True)

      if not response.ok:
          print(link, response)

      for block in response.iter_content(1024):
          if not block:
              break

          handle.write(block)
df['filepaths'] = file_paths

In [ ]:
import os

print(len(os.listdir('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/images')))

28285


In [ ]:
df.to_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/all_podcast_withpaths.csv')
df

,authors,Name,Artwork,Genre IDs,Primary Genre,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description,filepaths
0,Firearms Radio Network,Firearms Radio Network (All Shows),https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['Wilderness', 'Podcasts', 'Sports', 'Leisure'...",Wilderness,0,0,https://podcasts.apple.com/us/podcast/firearms...,http://feeds.feedburner.com/FRNetwork,0,Master Feed,/content/drive/MyDrive/Colab_Notebooks/podcast...
1,Jacob Mchangama,Clear and Present Danger - A history of free s...,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['History', 'Podcasts', 'Society & Culture']",History,0,0,https://podcasts.apple.com/us/podcast/clear-an...,https://freespeechhistory.libsyn.com/rss,0,"Why have kings, emperors, and governments kill...",/content/drive/MyDrive/Colab_Notebooks/podcast...
2,Kyric McMillan and Nick Vassolo,2 Idiots and a List,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['Music Commentary', 'Podcasts', 'Music']",Music Commentary,0,0,https://podcasts.apple.com/us/podcast/2-idiots...,https://anchor.fm/s/4a47e010/podcast/rss,0,"Welcome to “2 Idiots and a List”, the podcast ...",/content/drive/MyDrive/Colab_Notebooks/podcast...
3,Kate Spencer & Doree Shafrir,Forever35,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"['Fashion & Beauty', 'Podcasts', 'Arts', 'Heal...",Fashion & Beauty,0,0,https://podcasts.apple.com/us/podcast/forever3...,https://rss.acast.com/forever35,0,Which night creams will make you look well-res...,/content/drive/MyDrive/Colab_Notebooks/podcast...
4,Maverick Alexander,DarbyCast,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"['Comedy Fiction', 'Podcasts', 'Fiction']",Comedy Fiction,0,0,https://podcasts.apple.com/us/podcast/darbycas...,https://anchor.fm/s/1c855d9c/podcast/rss,0,"Neither the Darbyshire book series, nor this p...",/content/drive/MyDrive/Colab_Notebooks/podcast...
...,...,...,...,...,...,...,...,...,...,...,...,...
29274,NaN,Within the Trenches,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"['1304', '26']",Education,157,0,https://itunes.apple.com/us/podcast/within-the...,http://www.thejabberlog.com/category/within-th...,http://www.thejabberlog.com/category/within-th...,The JCast on Jabber Log features two main show...,/content/drive/MyDrive/Colab_Notebooks/podcast...
29275,NaN,"MTG Pro Tutor - Insights, Tips & Advice from M...",https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['1461', '26', '1323', '1460']",Games,229,"[55.63, 48.43, 61.8, 52.75, 43.17, 42.68, 57.5...",https://itunes.apple.com/us/podcast/mtg-pro-tu...,http://mtgprotutor.libsyn.com/rss,http://www.mtgprotutor.com,MTG Pro Tutor is a top rated Magic: The Gather...,/content/drive/MyDrive/Colab_Notebooks/podcast...
29276,NaN,National CASA Podcast,https://is4-ssl.mzstatic.com/image/thumb/Featu...,"['1475', '26', '1325', '1304', '1468']",Government,100,"[25.33, 26.57, 31.58, 2.23, 16.13, 1.72, 1.92,...",https://itunes.apple.com/us/podcast/national-c...,http://feeds.feedburner.com/NationalCasaPodcast,http://www.casaforchildren.org,"Today, more than 1,000 CASA program offices op...",/content/drive/MyDrive/Colab_Notebooks/podcast...
29277,NaN,Tottenham Hotspur Family,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['1467', '26', '1316']",Sports,112,"[64.53, 60.5, 76.18, 74.55, 88.47, 75.98, 73.5...",https://itunes.apple.com/us/podcast/tottenham-...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/javad-movahedi-1,The Tottenham Hotspur Family Podcast\n\n\nA gl...,/content/drive/MyDrive/Colab_Notebooks/podcast...


In [ ]:
df

,authors,Name,Artwork,Genre IDs,Primary Genre,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description,filepaths
0,Firearms Radio Network,Firearms Radio Network (All Shows),https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['Wilderness', 'Podcasts', 'Sports', 'Leisure'...",Wilderness,0,0,https://podcasts.apple.com/us/podcast/firearms...,http://feeds.feedburner.com/FRNetwork,0,Master Feed,/content/drive/MyDrive/Colab_Notebooks/podcast...
1,Jacob Mchangama,Clear and Present Danger - A history of free s...,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['History', 'Podcasts', 'Society & Culture']",History,0,0,https://podcasts.apple.com/us/podcast/clear-an...,https://freespeechhistory.libsyn.com/rss,0,"Why have kings, emperors, and governments kill...",/content/drive/MyDrive/Colab_Notebooks/podcast...
2,Kyric McMillan and Nick Vassolo,2 Idiots and a List,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['Music Commentary', 'Podcasts', 'Music']",Music Commentary,0,0,https://podcasts.apple.com/us/podcast/2-idiots...,https://anchor.fm/s/4a47e010/podcast/rss,0,"Welcome to “2 Idiots and a List”, the podcast ...",/content/drive/MyDrive/Colab_Notebooks/podcast...
3,Kate Spencer & Doree Shafrir,Forever35,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"['Fashion & Beauty', 'Podcasts', 'Arts', 'Heal...",Fashion & Beauty,0,0,https://podcasts.apple.com/us/podcast/forever3...,https://rss.acast.com/forever35,0,Which night creams will make you look well-res...,/content/drive/MyDrive/Colab_Notebooks/podcast...
4,Maverick Alexander,DarbyCast,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"['Comedy Fiction', 'Podcasts', 'Fiction']",Comedy Fiction,0,0,https://podcasts.apple.com/us/podcast/darbycas...,https://anchor.fm/s/1c855d9c/podcast/rss,0,"Neither the Darbyshire book series, nor this p...",/content/drive/MyDrive/Colab_Notebooks/podcast...
...,...,...,...,...,...,...,...,...,...,...,...,...
29274,NaN,Within the Trenches,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"['1304', '26']",Education,157,0,https://itunes.apple.com/us/podcast/within-the...,http://www.thejabberlog.com/category/within-th...,http://www.thejabberlog.com/category/within-th...,The JCast on Jabber Log features two main show...,/content/drive/MyDrive/Colab_Notebooks/podcast...
29275,NaN,"MTG Pro Tutor - Insights, Tips & Advice from M...",https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['1461', '26', '1323', '1460']",Games,229,"[55.63, 48.43, 61.8, 52.75, 43.17, 42.68, 57.5...",https://itunes.apple.com/us/podcast/mtg-pro-tu...,http://mtgprotutor.libsyn.com/rss,http://www.mtgprotutor.com,MTG Pro Tutor is a top rated Magic: The Gather...,/content/drive/MyDrive/Colab_Notebooks/podcast...
29276,NaN,National CASA Podcast,https://is4-ssl.mzstatic.com/image/thumb/Featu...,"['1475', '26', '1325', '1304', '1468']",Government,100,"[25.33, 26.57, 31.58, 2.23, 16.13, 1.72, 1.92,...",https://itunes.apple.com/us/podcast/national-c...,http://feeds.feedburner.com/NationalCasaPodcast,http://www.casaforchildren.org,"Today, more than 1,000 CASA program offices op...",/content/drive/MyDrive/Colab_Notebooks/podcast...
29277,NaN,Tottenham Hotspur Family,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['1467', '26', '1316']",Sports,112,"[64.53, 60.5, 76.18, 74.55, 88.47, 75.98, 73.5...",https://itunes.apple.com/us/podcast/tottenham-...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/javad-movahedi-1,The Tottenham Hotspur Family Podcast\n\n\nA gl...,/content/drive/MyDrive/Colab_Notebooks/podcast...
